In [77]:
import crocoddyl
import pinocchio
from scipy.stats import multivariate_normal
import mim_solvers
import numpy as np

In [65]:
from matplotlib import animation
from matplotlib import pyplot as plt


def animatePointMass(xs, sleep=50, show=False):
    print("processing the animation ... ")
    mass_size = 1.0
    fig = plt.figure()
    ax = plt.axes(xlim=(-2, 12), ylim=(-2, 12))
    patch = plt.Circle((0, 0), radius=0.2, fc="b")
    time_text = ax.text(0.02, 0.95, "", transform=ax.transAxes)

    def init():
        ax.add_patch(patch)
        time_text.set_text("")
        return patch, time_text

    def animate(i):
        x_pm = xs[i][0]
        y_pm = xs[i][1]
        patch.set_center((x_pm, y_pm))
        time = i * sleep / 1000.0
        time_text.set_text(f"time = {time:.1f} sec")
        return patch, time_text

    anim = animation.FuncAnimation(
        fig, animate, init_func=init, frames=len(xs), interval=sleep, blit=True
    )
    print("... processing done")
    if show:
        plt.show()
    return anim

In [68]:
class DifferentialActionModelPointMass(crocoddyl.DifferentialActionModelAbstract):
    def __init__(self):
        crocoddyl.DifferentialActionModelAbstract.__init__(
            self, crocoddyl.StateVector(4), 2, 4
        )  # nu = 2 {Fx, Fy}; nr = 3 {3 Gaussians, 1 Translation}
        
        self.unone = np.zeros(self.nu)
        self.m = 1.0
        self.costWeights = [
            120.0,
            40.0,
            200.0,
            10
        ]  # G1, G2, G3

    def calc(self, data, x, u=None):
        if u is None:
            u = self.unone
        # Getting the state and control variables
        X, Y, Xdot, Ydot = x[0], x[1], x[2], x[3]
        fx = u[0]
        fy = u[1]

        # Shortname for system parameters
        m = self.m

        # Defining the equation of motions
        xddot = fx / m
        yddot = fy / m
        data.xout = np.matrix([xddot, yddot]).T

        # Cost parameters (Gaussians)
        goal = np.array([10, 10])
        mu1 = np.array([7, 2])
        S1 = np.array([[2, 0], [0, 1]])*10
        mu2 = np.array([2, 7])
        S2 = np.array([[2, 1], [1, 1]])*10
        mu3 = np.array([5, 9])
        S3 = np.array([[3, 0], [0 , 1]])*10

        G1 = multivariate_normal(mu1,S1)
        G2 = multivariate_normal(mu2,S2)
        G3 = multivariate_normal(mu3,S3)

        translation = np.sqrt(np.sum((goal - x[0:2])*(goal - x[0:2])))

        # Computing the cost residual and value
        data.r = np.matrix(self.costWeights * np.array([G1.pdf(x[0:2]), G2.pdf(x[0:2]), G3.pdf(x[0:2]), translation])).T
        data.cost = 0.5 * sum(np.asarray(data.r) ** 2)


In [69]:
PM_DAM = DifferentialActionModelPointMass()
pmData = PM_DAM.createData()
x = np.array([0, 0, 0, 0])
u = np.zeros(2)
PM_DAM.calc(pmData, x, u)

In [70]:
pmData.r

array([3.24800540e-01, 1.57394858e-02, 2.11076952e-02, 1.41421356e+02])

In [71]:
PM_ND = crocoddyl.DifferentialActionModelNumDiff(PM_DAM, True)

In [72]:
timeStep = 5e-2
PM_IAM = crocoddyl.IntegratedActionModelEuler(PM_ND, timeStep)

In [86]:
# Fill the number of knots (T) and the time step (dt)
x0 = np.array([0.0, 0.0, 0.0, 0.0])
u0 = np.array([0.0, 0.0])
T = 50
problem = crocoddyl.ShootingProblem(x0, [PM_IAM] * T, PM_IAM)

In [ ]:
# Creating the DDP solver
ddp = crocoddyl.SolverDDP(problem)
ddp.setCallbacks([crocoddyl.CallbackVerbose()])

# Solving this problem
done = ddp.solve([], [], 1000)
print(done)
print(ddp.us)

In [ ]:
# Creating the SQP solver
sqp = mim_solvers.SolverSQP(problem)
sqp.setCallbacks([crocoddyl.CallbackVerbose()])
sqp.with_callbacks=True
xs_init = [x0 for i in range(T+1)]
us_init = [u0 for i in range(T)]

# Solving this problem
done = sqp.solve(xs_init, us_init, 1000)
print(done)
print(sqp.us)

iter     merit         cost         grad      step    ||gaps||        KKT
   1  3.50002e+04  3.50002e+04  5.92852e+02  0.1250  0.00000e+00  9.99978e+02
   2  3.12841e+04  3.12841e+04  4.52879e+02  1.0000  1.86517e-14  1.23394e+03
   3  1.38919e+04  1.38919e+04  3.53654e+02  1.0000  1.47968e-06  8.24627e+02
   4  1.10485e+04  1.10485e+04  3.72564e+02  0.5000  1.02794e-06  7.40221e+02
   5  2.69516e+04  2.69516e+04  4.26114e+02  1.0000  8.91840e-07  1.27479e+03
   6  2.37909e+06  2.37909e+06  2.39665e+03  0.5000  7.48008e-07  1.74379e+04
   7  7.45763e+07  7.45763e+07  1.29020e+04  0.0156  7.13560e-07  1.00827e+05
   8  7.41747e+07  7.41747e+07  3.28181e+04  0.0020  7.07709e-07  9.66282e+04
   9  7.40643e+07  7.40643e+07  5.70501e+03  0.1250  7.10912e-07  9.48607e+04
  10  7.32121e+07  7.32121e+07  5.90342e+03  0.0625  1.19539e-06  7.68411e+04
iter     merit         cost         grad      step    ||gaps||        KKT
  11  6.92132e+07  6.92132e+07  2.88678e+03  0.2500  1.24848e-06  8.8914